In [11]:
import requests
obj = requests.get('http://api.conceptnet.io/query?start=/c/en/apple&rel=/r/ExternalURL&limit=1000')
obj.content



b'<html>\r\n<head><title>502 Bad Gateway</title></head>\r\n<body bgcolor="white">\r\n<center><h1>502 Bad Gateway</h1></center>\r\n<hr><center>nginx/1.14.0 (Ubuntu)</center>\r\n</body>\r\n</html>\r\n'

In [ ]:
### Issue https://github.com/commonsense/conceptnet5/issues/341

import requests
from gradio_client import Client

class ConceptNetWithFallback:
    def __init__(self):
        self.conceptnet_url = "http://api.conceptnet.io"
        self.fallback_space = "cstr/conceptnet-normalized"
        self.fallback_client = None


    def _init_fallback(self):
        """Initialize Gradio client only when needed"""
        if self.fallback_client is None:
            self.fallback_client = Client("https://cstr-conceptnet-normalized.hf.space")

    def query(self, word, language="en", relations=['IsA', 'RelatedTo', 'PartOf', 'HasA']):
        """
        Query ConceptNet with automatic fallback
        """
        # Try ConceptNet first
        # try:
        #     response = requests.get(
        #         f"{self.conceptnet_url}/c/{language}/{word}",
        #         timeout=10,
        #         relations=relations
        #     )
            
        #     # Check for 502 or other server errors
        #     if response.status_code == 502:
        #         print(f"⚠️ ConceptNet returned 502, using fallback...")
        #         return self._fallback_query(word, language)
            
        #     if response.status_code == 200:
        #         return {
        #             "source": "conceptnet",
        #             "data": response.json()
        #         }
        #     else:
        #         print(f"⚠️ ConceptNet returned {response.status_code}, using fallback...")
        return self._fallback_query(word, language)
                
        # except requests.exceptions.RequestException as e:
        #     print(f"⚠️ ConceptNet connection error: {e}, using fallback...")
        #     return self._fallback_query(word, language, relations)
        

    def _fallback_query(self, word, language="en", relations=['IsA', 'RelatedTo', 'PartOf', 'HasA']):
        """
        Query the normalized ConceptNet via Hugging Face
        """
        try:
            self._init_fallback()
            
            # Call the Gradio interface
            # The function is called "get_semantic_profile" based on the UI
            result = self.fallback_client.predict(
                word=word,
                lang=language,
                api_name="/get_semantic_profile",
                selected_relations=relations
            )
            
            return {
                "source": "conceptnet_normalized",
                "data": result
            }
            
        except Exception as e:
            print(f"❌ Fallback also failed: {e}")
            return {
                "source": "error",
                "error": str(e)
            }
    

SyntaxError: unterminated string literal (detected at line 411) (97710467.py, line 411)

In [29]:
cn = ConceptNetWithFallback()
result = cn.query("politics", "en")

print(f"Data source: {result['source']}")
print(f"Result: {result['data']}")



Loaded as API: https://cstr-conceptnet-normalized.hf.space/ ✔
Data source: conceptnet_normalized
Result: # 🧠 Semantic Profile: 'politics' (EN)

**Node:** `http://conceptnet.io/c/en/politics` → **politics**

## IsA

- **politics** IsA → *what politicians do* `[1.000]`

## RelatedTo

- *politic* RelatedTo → **politics** `[2.000]`
- *political* RelatedTo → **politics** `[2.000]`
- *politically* RelatedTo → **politics** `[2.000]`
- *politician* RelatedTo → **politics** `[2.000]`
- *تدبير* RelatedTo → **politics** `[1.000]`
- *سياسة* RelatedTo → **politics** `[1.000]`
- *kulturpolitisch* RelatedTo → **politics** `[1.000]`

## PartOf

- *influencing public decisions* PartOf → **politics** `[1.000]`

## HasA

*No results*

---
**Total relations:** 9
